# very basic threads
The following shows threading


In [11]:
from threading import Thread
import time

def test_function(*args):
    print("From thread", ''.join(args))
    time.sleep(5)

th1 = Thread(target= test_function, args='th 1')
th2 = Thread(target= test_function, args='th 2')

th1.start()
th2.start()

print("from main")

th1.join()
th2.join()

From thread th 1
From thread th 2
from main


Looking at semaphore, the analogy is that it controls the number of threads access
## mutex
mutex mutual exclusion  concurrency control, which is instituted for the purpose of preventing race conditions. It is the requirement that one thread of execution never enters a critical section while a concurrent thread of execution is already accessing said critical section. Python locks in p3 use lock